# In questo notebook calcolo il numero di tweet positivi, negativi, neutrali e totali di ciascun utente (relativi al medesimo argomento)

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import csv

In [3]:
spark = SparkSession.builder.appName("sentiment analysis").config("spark.executor.memory", "500m").getOrCreate()

In [4]:
df = spark.read.options(sep=",", header=True, quote="\"", escape="\"", multiline=True).csv('Sentiment_analysis_war.csv')

In [5]:
tuples_RDD = df.rdd

In [6]:
def count_result(sentiment):
    if sentiment == 'Positive':
        return (1,0,0)
    elif sentiment == 'Neutral':
        return (0,1,0)
    else:
        return (0,0,1)

In [7]:
user_sentiment_RDD = tuples_RDD.map(f=lambda e: (e[2], count_result(e[5])))

In [8]:
user_sentiment_to_one_RDD = user_sentiment_RDD.map(f=lambda x: (x[0], (x[1][0], x[1][1], x[1][2], 1)))

In [9]:
user_to_sum_RDD = user_sentiment_to_one_RDD.reduceByKey(func=lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2], a[3]+b[3]))

In [10]:
result_RDD = user_to_sum_RDD.map(f=lambda x: x[0] + "," + str(x[1][0]) + "," + str(x[1][1]) + "," + str(x[1][2]) + "," + str(x[1][3]))

In [11]:
result_RDD.saveAsTextFile('Somma_sentiment_analysis_war')